In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats

In [2]:
available_stocks = pd.read_json('curated_stocks_all.json')
stocks_10_to_30 = pd.read_json('stocks_10_to_30.json')
stocks_all = pd.read_json('stocks_all.json')

In [3]:
available_stocks_list = available_stocks['symbol'].tolist()

In [4]:
stocks_10_to_30 = stocks_10_to_30[stocks_10_to_30['Symbol'].isin(available_stocks_list)]
stocks_all = stocks_all[stocks_all['Symbol'].isin(available_stocks_list)]

In [5]:
stocks_10_to_30.drop(columns=['EBITDA', 'Gross_Profit'], inplace=True)
stocks_all.drop(columns=['EBITDA', 'Gross_Profit'], inplace=True)

In [6]:
stocks_all

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit
0,A,"Agilent Technologies, Inc.",143.9300,31.974632,7.884889,6.206338,4.688880e+10,1147878,24.294715,12.720781
1,AA,Alcoa Corporation,55.5200,NaN,1.871842,0.761471,9.847040e+09,6162432,4.255419,5.094175
9,AAIC,Arlington Asset Investment Corp,3.0400,46.666667,0.491369,10.280004,8.506130e+08,26331,69.854069,96.310349
16,AAME,Atlantic American Corporation,2.3899,20.514950,0.356402,0.252528,9.692350e+07,8813,NaN,NaN
17,AAN,"Aarons Holdings Company, Inc.",12.7800,NaN,0.642157,0.169820,1.078720e+09,854792,1.269855,0.936029
...,...,...,...,...,...,...,...,...,...,...
7768,ZUO,"Zuora, Inc.",10.0200,NaN,NaN,3.195704,9.353660e+08,1674581,-12.079214,3.832666
7769,ZVIA,Zevia PBC,3.7500,NaN,NaN,0.946400,1.121920e+08,100352,-2.436943,1.633690
7773,ZYME,Zymeworks Inc.,8.0100,NaN,1.497712,15.458431,2.726860e+08,311630,-1.247545,NaN
7774,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4962,NaN,0.284225,NaN,-2.351970e+07,253629,0.668664,95.167516


In [7]:
stocks_10_to_30 = stocks_10_to_30[(stocks_10_to_30['P/E'].isna() == False) &
               (stocks_10_to_30['P/B'].isna() == False) &
               (stocks_10_to_30['P/S'].isna() == False) &
               (stocks_10_to_30['EV/EBITDA'].isna() == False) &
               (stocks_10_to_30['EV/Gross_Profit'].isna() == False)]

In [8]:
stocks_all = stocks_all[(stocks_all['P/E'].isna() == False) &
               (stocks_all['P/B'].isna() == False) &
               (stocks_all['P/S'].isna() == False) &
               (stocks_all['EV/EBITDA'].isna() == False) &
               (stocks_all['EV/Gross_Profit'].isna() == False)]

In [9]:
stocks_10_to_30 = stocks_10_to_30.reset_index().drop(columns='index')
stocks_all = stocks_all.reset_index().drop(columns='index')

In [10]:
stocks_all

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit
0,A,"Agilent Technologies, Inc.",143.93,31.974632,7.884889,6.206338,4.688880e+10,1147878,24.294715,12.720781
1,AAIC,Arlington Asset Investment Corp,3.04,46.666667,0.491369,10.280004,8.506130e+08,26331,69.854069,96.310349
2,AAON,"AAON, Inc.",93.92,49.362438,8.704841,5.560728,4.200870e+09,257297,25.902675,18.302690
3,AAP,Advance Auto Parts Inc.,138.48,16.837815,3.080387,0.757743,1.256320e+10,1907651,12.107860,2.544567
4,AAPL,Apple Inc.,151.03,24.790173,45.909057,6.040911,2.132880e+12,70730970,17.023817,12.488904
...,...,...,...,...,...,...,...,...,...,...
1462,ZM,"Zoom Video Communications, Inc.",70.81,217.657935,3.343647,4.891431,1.664450e+10,4358032,30.080657,5.487546
1463,ZTO,ZTO Express (Cayman) Inc.,24.72,20.216938,1.912493,4.214114,1.880120e+10,4381613,13.428984,18.401690
1464,ZTS,Zoetis Inc.,170.56,37.527269,19.207456,9.814047,7.295950e+10,1663200,21.502947,13.323503
1465,ZUMZ,Zumiez Inc.,23.55,10.632980,1.060048,0.486099,5.491020e+08,329446,3.548340,1.252217


In [11]:
# new_columns = ['PE Percentile','PS Percentile','PS Percentile', 'EV/EBITDA Percentile', 'EV/Gross_Profit Percentile']

# for col in new_columns:
#     stocks_10_to_30[col] = [np.NaN] * len(stocks_10_to_30)

In [12]:
stocks_10_to_30

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit
0,ABR,Arbor Realty Trust,15.0700,9.274518,1.158380,2.591772,1.586110e+10,3478155,16.544003,13.923004
1,ACI,"Albertsons Companies, Inc.",20.0800,8.734834,3.207632,0.131376,2.265800e+10,5447924,4.553731,1.180947
2,ACIW,"ACI Worldwide, Inc.",25.8700,20.607117,2.354861,2.088885,3.593990e+09,748280,10.426459,5.998461
3,ACRE,Ares Commercial Real Estate Corporation,11.5500,18.706352,0.819347,3.392792,2.160160e+09,584255,15.342808,13.998471
4,AGRO,Adecoagro S.A.,8.0500,5.536545,0.888362,0.892094,2.074830e+09,475535,12.818433,10.010663
...,...,...,...,...,...,...,...,...,...,...
212,WU,Western Union Company (The),12.8300,5.490330,10.052765,1.117435,6.737000e+09,5506871,6.201215,4.059167
213,XP,XP Inc.,11.4900,9.180093,2.485043,2.853403,1.942070e+10,5350907,23.185552,11.879771
214,ZEV,"Lightning eMotors, Inc",0.6722,1.135284,1.836370,1.770601,1.028380e+08,1264956,-1.554054,-17.159686
215,ZI,ZoomInfo Technologies Inc.,25.8500,158.237548,4.407425,9.103884,1.293470e+10,3889386,47.694322,14.569385


In [13]:
rv_dataframe = stocks_all
rv_dataframe = rv_dataframe.reset_index().drop(columns='index')
metrics = {
    'P/E':'PE Percentile',
    'P/B':'PB Percentile',
    'P/S':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Gross_Profit':'EV/Gross_Profit Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] \
            = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/Gross_Profit Percentile
0,A,"Agilent Technologies, Inc.",143.93,31.974632,7.884889,6.206338,4.688880e+10,1147878,24.294715,12.720781,73.892297,88.275392,87.389230,90.184049,77.982277
1,AAIC,Arlington Asset Investment Corp,3.04,46.666667,0.491369,10.280004,8.506130e+08,26331,69.854069,96.310349,85.071575,1.635992,94.887526,98.364008,99.931834
2,AAON,"AAON, Inc.",93.92,49.362438,8.704841,5.560728,4.200870e+09,257297,25.902675,18.302690,86.366735,89.706885,85.412406,92.024540,88.888889
3,AAP,Advance Auto Parts Inc.,138.48,16.837815,3.080387,0.757743,1.256320e+10,1907651,12.107860,2.544567,42.603954,60.190866,21.608725,55.623722,15.269257
4,AAPL,Apple Inc.,151.03,24.790173,45.909057,6.040911,2.132880e+12,70730970,17.023817,12.488904,62.644853,98.909339,86.775733,76.414451,76.959782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,ZM,"Zoom Video Communications, Inc.",70.81,217.657935,3.343647,4.891431,1.664450e+10,4358032,30.080657,5.487546,97.614179,63.735515,82.549421,94.137696,43.830948
1463,ZTO,ZTO Express (Cayman) Inc.,24.72,20.216938,1.912493,4.214114,1.880120e+10,4381613,13.428984,18.401690,52.079073,36.809816,78.595774,62.576687,89.025222
1464,ZTS,Zoetis Inc.,170.56,37.527269,19.207456,9.814047,7.295950e+10,1663200,21.502947,13.323503,79.141104,96.591684,94.274029,86.912065,79.481936
1465,ZUMZ,Zumiez Inc.,23.55,10.632980,1.060048,0.486099,5.491020e+08,329446,3.548340,1.252217,24.744376,12.338105,12.610770,8.793456,3.817314


In [14]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/Gross_Profit Percentile,RV Score
0,A,"Agilent Technologies, Inc.",143.93,31.974632,7.884889,6.206338,4.688880e+10,1147878,24.294715,12.720781,73.892297,88.275392,87.389230,90.184049,77.982277,83.544649
1,AAIC,Arlington Asset Investment Corp,3.04,46.666667,0.491369,10.280004,8.506130e+08,26331,69.854069,96.310349,85.071575,1.635992,94.887526,98.364008,99.931834,75.978187
2,AAON,"AAON, Inc.",93.92,49.362438,8.704841,5.560728,4.200870e+09,257297,25.902675,18.302690,86.366735,89.706885,85.412406,92.024540,88.888889,88.479891
3,AAP,Advance Auto Parts Inc.,138.48,16.837815,3.080387,0.757743,1.256320e+10,1907651,12.107860,2.544567,42.603954,60.190866,21.608725,55.623722,15.269257,39.059305
4,AAPL,Apple Inc.,151.03,24.790173,45.909057,6.040911,2.132880e+12,70730970,17.023817,12.488904,62.644853,98.909339,86.775733,76.414451,76.959782,80.340832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,ZM,"Zoom Video Communications, Inc.",70.81,217.657935,3.343647,4.891431,1.664450e+10,4358032,30.080657,5.487546,97.614179,63.735515,82.549421,94.137696,43.830948,76.373551
1463,ZTO,ZTO Express (Cayman) Inc.,24.72,20.216938,1.912493,4.214114,1.880120e+10,4381613,13.428984,18.401690,52.079073,36.809816,78.595774,62.576687,89.025222,63.817314
1464,ZTS,Zoetis Inc.,170.56,37.527269,19.207456,9.814047,7.295950e+10,1663200,21.502947,13.323503,79.141104,96.591684,94.274029,86.912065,79.481936,87.280164
1465,ZUMZ,Zumiez Inc.,23.55,10.632980,1.060048,0.486099,5.491020e+08,329446,3.548340,1.252217,24.744376,12.338105,12.610770,8.793456,3.817314,12.460804


In [15]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(inplace=True, drop=True)

In [16]:
rv_dataframe[:50]

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/Gross_Profit Percentile,RV Score
0,GLBS,Globus Maritime Limited,1.1800,0.635060,0.163064,0.396004,6.421700e+06,57345,0.142126,0.294790,0.136333,0.272665,8.929789,1.363327,1.431493,2.426721
1,GURE,"Gulf Resources, Inc.",3.2300,2.879770,0.116907,0.608937,-4.796460e+07,5969,-0.972675,-1.759919,2.249489,0.136333,16.973415,1.158828,0.749830,4.253579
2,SBFM,Sunshine Biopharma Inc.,0.9195,0.017795,0.105599,0.052035,-2.596580e+07,75677,6.645288,-265.349752,0.068166,0.068166,0.204499,24.744376,0.068166,5.030675
3,PBF,PBF Energy Inc.,48.0700,2.084774,1.244792,0.128214,5.952110e+09,1991280,1.254396,1.265706,1.363327,17.177914,1.022495,2.044990,4.089980,5.139741
4,SCX,L.S. Starrett Company (The),11.5000,5.665870,0.776044,0.319523,7.767080e+07,44809,2.564832,0.921952,9.747785,5.180641,6.339468,4.839809,2.249489,5.671438
5,VTLE,"Vital Energy, Inc.",57.0600,1.452949,0.820943,0.477700,1.946270e+09,856358,1.379723,1.700561,0.681663,7.157464,12.133606,2.317655,7.157464,5.889571
6,BXC,Bluelinx Holdings Inc.,88.3500,2.823095,1.323178,0.182207,9.630910e+08,90883,2.081810,1.195835,2.181322,19.904567,2.113156,3.612815,3.271984,6.216769
7,BZH,"Beazer Homes USA, Inc.",15.5000,2.227299,0.500047,0.202162,1.226800e+09,256271,4.491190,2.285148,1.499659,1.840491,2.453988,13.224267,12.201772,6.244035
8,X,United States Steel Corporation,31.3700,3.457770,0.693375,0.407503,7.296710e+09,8620286,1.932391,2.178122,3.340150,3.680982,9.406953,3.203817,11.656442,6.257669
9,ASTL,Algoma Steel Group Inc.,9.0600,4.036504,1.073403,0.446339,5.880975e+08,2656893,1.188925,0.486164,4.567144,12.747103,10.702113,1.908657,1.704158,6.325835


In [17]:
# rv_dataframe.to_csv('10_to_30_vol_over_500k_result.csv', index=False)

In [18]:
'Talos', 'HOUS', 'TK'

('Talos', 'HOUS', 'TK')